In [ ]:
using Plots;
gr();#plotlyjs();
using Revise
using FUSE
FUSE.logging(Logging.Info; actors=Logging.Error);

In [ ]:
#ini,act = FUSE.case_parameters(:ITER; init_from=:ods);
#ini,act = FUSE.case_parameters(:ITER; init_from=:scalars);
#ini,act = FUSE.case_parameters(:D3D);
ini, act = FUSE.case_parameters(:FPP; version=:v1_demount, init_from=:scalars);
#ini,act = FUSE.case_parameters(:FPP; version=:v1, init_from=:scalars);
#ini,act = FUSE.case_parameters(:CAT);
#ini,act = FUSE.case_parameters(:HDB5; tokamak=:JET, case=500);

# Modify default settings
#ini.core_profiles.zeff = 2.0
#ini.equilibrium.ζ = 0.1
#act.ActorTauenn.transport_model = :ds03
act.ActorEquilibrium.model = :CHEASE;

In [ ]:
# initialize data structure
#ini.equilibrium.R0=10#4.824432302041749*1.5
#ini.equilibrium.ϵ=0.2857142857142857*1.5
dd = FUSE.init(ini, act; do_plot=true);

In [ ]:
dd_copy = deepcopy(dd);

In [ ]:
dd = deepcopy(dd_copy)
act.ActorTauenn.do_plot = true
FUSE.ActorWholeFacility(dd, act);

In [ ]:
IMAS.freeze(dd.costing)

In [ ]:
dd = deepcopy(dd_copy)
FUSE.ActorEquilibriumTransport(dd, act; do_plot=true);

In [ ]:
@time FUSE.ActorHFSsizing(dd, act; do_plot=true);
display(plot(dd.solid_mechanics.center_stack.stress))
dd.build.oh

In [ ]:
@time FUSE.ActorLFSsizing(dd, act; do_plot=true);
display(dd.build.tf)

In [ ]:
@time FUSE.ActorCXbuild(dd, act; do_plot=true);

In [ ]:
dd_copy = deepcopy(dd);

In [ ]:
dd = deepcopy(dd_copy)
ini.pf_active.n_pf_coils_outside = 5
FUSE.init_pf_active(dd, ini, act)
# λ_currents: make bigger to force currents to fit within the current limits
# update_equilibrium: overwrite the target equilibrium with what the coils can actually generate
@time FUSE.ActorPFcoilsOpt(dd, act; weight_lcfs=1, weight_null=1E-3, weight_currents=1.0, weight_strike=1.0, do_plot=true, update_equilibrium=true);

In [ ]:
@time FUSE.ActorCXbuild(dd, act, rebuild_wall=true);
@time FUSE.ActorPFcoilsOpt(dd, act; weight_lcfs=1, weight_null=1E-3, weight_currents=1.0, weight_strike=1.0, do_plot=true, update_equilibrium=false);

In [ ]:
@time FUSE.ActorNeutronics(dd, act; do_plot=true);

In [ ]:
@time FUSE.ActorBlanket(dd, act)#; do_plot=true);
dd.blanket

In [ ]:
@time FUSE.ActorDivertors(dd, act)#; do_plot=true);
dd.divertors

In [ ]:
@time FUSE.ActorBalanceOfPlant(dd, act)#; do_plot=true);
IMAS.freeze(dd.balance_of_plant)

In [ ]:
@time FUSE.ActorCosting(dd, act)#; do_plot=true);
IMAS.freeze(dd.costing)

In [ ]:
# Save `dd` to Json, so we can share our results with others
IMAS.imas2json(dd, "fpp_v1_FUSE.json")

In [ ]:
plot(dd.equilibrium; cx=true)
plot!(dd.build; legend=false)
display(plot!(dd.pf_active))

display(dd.build.layer)

display(plot(dd.equilibrium))

display(plot(dd.core_profiles))

display(plot(dd.core_sources))

display(IMAS.freeze(dd.balance_of_plant))

display(IMAS.freeze(dd.costing))